# Multiple Hypothesis Testing Correction

In [29]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_theme(
    context="paper", 
    style="whitegrid", 
    font_scale=1.2,
    rc={'figure.figsize': (10, 10), 'figure.dpi': 300}
)

from common.paths import PLS_RESULTS, RIDGE_RESULTS, CROSS_PRED_RESULTS
from common.results import load_results

from multipy.fwer import bonferroni
from multipy.fdr import lsu

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get data

### Manual Input

In [2]:
# bin_all = [0.0003, 0.0003, 0.0003, 0.0003, 0.0100, 0.0643]
# bin_1 = [0.0953, 0.0313, 0.0593, 0.6485, 0.6048, 0.8287]
# bin_2 = [0.0003, 0.0247, 0.0003, 0.0020, 0.0107, 0.4159]
# bin_3 = [0.1789, 0.0653, 0.3196, 0.4172, 0.2779, 0.3162]
# pvals = np.array(bin_all + bin_1 + bin_2 + bin_3)
pvals = np.array([0.3214, 0.0140, 0.0200, 0.5309, 0.1497, 0.4950])
df = pd.DataFrame({'Features': ['Feature {}'.format(i) for i in range(1, len(pvals) + 1)], 'P-value':pvals})
display(df)

,Features,P-value
0,Feature 1,0.3214
1,Feature 2,0.0140
2,Feature 3,0.0200
3,Feature 4,0.5309
4,Feature 5,0.1497
5,Feature 6,0.4950


### Within-Prediction

In [13]:
model = 'ridge'
population = 'healthy'

fn = f'{model}_pts_{population}'
results_folder = RIDGE_RESULTS if model == 'ridge' else PLS_RESULTS
df, results_path = load_results(fn, results_folder)
pvals = df['P-value']
display(df)

,Model,Population,Target,Bin,Alpha,Score,P-value
0,ridge,healthy,WISC_FSIQ,All,1,0.039798,0.417166
1,ridge,healthy,WISC_VSI,All,1,0.161814,0.107784
2,ridge,healthy,WISC_VCI,All,9901,0.196705,0.051896
3,ridge,healthy,WISC_FRI,All,1,-0.072839,0.734531
4,ridge,healthy,WISC_WMI,All,9901,0.116987,0.213573
5,ridge,healthy,WISC_PSI,All,1,-0.056080,0.698603


### Cross-Prediction

In [6]:
# fn = 'ridge_pts_age_cross_prediction'
fn = 'ridge_pts_age_cross_prediction_bin_two_equal_samples'
df, results_path = load_results(fn, CROSS_PRED_RESULTS)
df = df.reset_index(drop=True)
pvals = df['P-value']
display(df)

,Model,Target,Train,Test,Score,P-value,Population,Num Permutations,FDR
0,ridge,WISC_FSIQ,Bin 1,Bin 1,0.230310,0.023952,adhd,500,NaN
1,ridge,WISC_FSIQ,Bin 1,Bin 2 Equal,0.350181,0.001996,adhd,500,NaN
2,ridge,WISC_FSIQ,Bin 1,Bin 3,0.207953,0.015968,adhd,500,NaN
3,ridge,WISC_FSIQ,Bin 2 Equal,Bin 2 Equal,0.213853,0.033932,adhd,500,NaN
4,ridge,WISC_FSIQ,Bin 2 Equal,Bin 3,0.121881,0.079840,adhd,500,NaN
5,ridge,WISC_FSIQ,Bin 2 Equal,Bin 1,0.270841,0.001996,adhd,500,NaN
6,ridge,WISC_FSIQ,Bin 3,Bin 3,0.090251,0.209581,adhd,500,NaN
7,ridge,WISC_FSIQ,Bin 3,Bin 1,0.256395,0.005988,adhd,500,NaN
8,ridge,WISC_FSIQ,Bin 3,Bin 2 Equal,0.220301,0.003992,adhd,500,NaN
9,ridge,WISC_VSI,Bin 1,Bin 1,0.229323,0.025948,adhd,500,NaN


## BFR/FDR Correction

In [14]:
a_threshold = 0.05
df['Uncorrected'] = pvals < a_threshold
df['BFR'] = bonferroni(pvals, alpha=a_threshold)
df['FDR'] = lsu(pvals, q=a_threshold)
display(df.round(4))

,Model,Target,Train,Test,Score,P-value,Population,Num Permutations,FDR,Uncorrected
0,ridge,WISC_FSIQ,Bin 1,Bin 1,0.2303,0.0240,adhd,500,False,True
1,ridge,WISC_FSIQ,Bin 1,Bin 2 Equal,0.3502,0.0020,adhd,500,True,True
2,ridge,WISC_FSIQ,Bin 1,Bin 3,0.2080,0.0160,adhd,500,False,True
3,ridge,WISC_FSIQ,Bin 2 Equal,Bin 2 Equal,0.2139,0.0339,adhd,500,False,True
4,ridge,WISC_FSIQ,Bin 2 Equal,Bin 3,0.1219,0.0798,adhd,500,False,False
5,ridge,WISC_FSIQ,Bin 2 Equal,Bin 1,0.2708,0.0020,adhd,500,True,True
6,ridge,WISC_FSIQ,Bin 3,Bin 3,0.0903,0.2096,adhd,500,False,False
7,ridge,WISC_FSIQ,Bin 3,Bin 1,0.2564,0.0060,adhd,500,True,True
8,ridge,WISC_FSIQ,Bin 3,Bin 2 Equal,0.2203,0.0040,adhd,500,True,True
9,ridge,WISC_VSI,Bin 1,Bin 1,0.2293,0.0259,adhd,500,False,True


### Save and update results file

In [15]:
df.to_csv(results_path)

### Get FDR q

In [18]:
m = len(pvals)
sort_ind = np.argsort(pvals)
qs = [(i+1.)*0.05/m for i, p in enumerate(pvals[sort_ind])]
a = pd.DataFrame([(p, q) for p, q in zip(pvals[sort_ind], qs)])
display(a)

,0,1
0,0.001996,0.000926
1,0.001996,0.001852
2,0.001996,0.002778
3,0.001996,0.003704
4,0.001996,0.004630
5,0.001996,0.005556
6,0.001996,0.006481
7,0.001996,0.007407
8,0.001996,0.008333
9,0.001996,0.009259


## Max Statistic Method

In [35]:
from common.binning import BIN_LABELS, ONLY_BIN_LABELS, EQUAL_BIN_LABELS
from common.paths import RIDGE_RESULTS, RIDGE_PSCORES, PLS_RESULTS, PLS_PSCORES
from common.results import save_perm_score, load_perm_score, load_results
from common.wisc import WISC_LEVEL

### Merge permutation scores per age bin

In [21]:
# Only need to run this code block once per model/population to generate merged permutation array
model = 'pls'
population = 'adhd'
pscores = RIDGE_PSCORES if model == 'ridge' else PLS_PSCORES

for bin_label in BIN_LABELS:
    bin_perm_scores = []
    
    for target in WISC_LEVEL[5]:
        perm_score = load_perm_score(f'{model}_{population}_{target}_{bin_label}_perm_scores', pscores)
        bin_perm_scores.append(perm_score)
    
    bin_perm_scores = np.concatenate(bin_perm_scores)
    print(bin_label, bin_perm_scores.shape)
    save_perm_score(bin_perm_scores, f'{model}_{population}_{bin_label}_perm_scores', pscores)

All (3000,)
Bin 1 (3000,)
Bin 2 (3000,)
Bin 3 (3000,)


### Load results (true scores)

In [25]:
model = 'pls'
population = 'adhd'
results = RIDGE_RESULTS if model == 'ridge' else PLS_RESULTS

df, results_path = load_results(f'{model}_pts_{population}', results)
# display(df)

In [ ]:
a_threshold = 0.05
bin_threshold_scores = {k: None for k in EQUAL_BIN_LABELS}
max_stat_temp = pd.Series(dtype='bool')
pscores = RIDGE_PSCORES if model == 'ridge' else PLS_PSCORES

for bin_label in BIN_LABELS:
    bin_perm_scores = load_perm_score(f'{model}_{population}_{bin_label}_perm_scores', pscores)
    threshold_index = int(bin_perm_scores.shape[0] * (1 - a_threshold))
    threshold_score = np.sort(bin_perm_scores)[threshold_index]
    bin_threshold_scores[bin_label] = threshold_score
    
    bin_results = df[df['Bin'] == bin_label]
    max_stat_temp = pd.concat([max_stat_temp, threshold_score < bin_results['Score']])

df['Max-stat'] = max_stat_temp
# df.sort_values('Bin')
display(df)

### Save and update results file

In [40]:
df.to_csv(results_path)

### Visualize

In [ ]:
for bin_label in BIN_LABELS:
    threshold_score = bin_threshold_scores[bin_label]
    
    g = sns.displot(bin_perm_scores, color='lightgrey', element="step")
    g.fig.subplots_adjust(top=0.9)
    g.fig.suptitle(f'Model: {model}, Population: {population}, {bin_label}, Num Perm: {bin_perm_scores.shape[0]}')
    g.ax.axvline(threshold_score, label='95th', color='black')
    
    for target_idx, target in enumerate(WISC_LEVEL[5]):
        target_score = df.loc[(df['Bin'] == bin_label) & (df['Target'] == target)]['Score']
        target_score = target_score.to_list()[0]
#         print(bin_label, target, target_score)
        g.ax.axvline(target_score, label=target, ls=(0, (3, np.random.randint(1, 6))), color=sns.color_palette()[target_idx])
    g.fig.legend(loc='center left', bbox_to_anchor=(1, 0.5))